## Acquaring and Loading Data

Presenting the code and methods for acquiring the data. Loading the data into appropriate format for analysis. Explaining the process and results

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Foursquare developer credentials, which will be hidden later.

In [2]:
CLIENT_ID = 'xxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxx' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxx
CLIENT_SECRET:xxxxx


Load open source rental data in Singapore provided by government

In [ ]:
rent_df = pd.read_csv('Singapore_median_rent.csv')

#examing head of rent csv file 
rent_df.head()

After loading the dataset in and from the head of dataset we do already see some missing values in the median rent column, which will require cleaning and handling of NaN values. These rows are dropped as they are insignificant and do not provide useful information for our analysis.

## Understanding, Wrangling and Cleaning Data

Presenting the code and methods for acquiring the data. Loading the data into appropriate format for analysis. Explaining the process and results

Notice that the old statistics may not be relevant in our study as the rental has increased significantly over the years. Hence, we filter the data and only uses information from year 2017 onwards such that the rental price will be closer to the current market price. Executive HDB type is removed as it is irrelevant.

In [ ]:
rent_df = rent_df[rent_df['quarter'] >= "2017-Q1"]
rent_df = rent_df[rent_df['flat_type'] != "EXEC"]

rent_df.head()

Also, notice that in median_rent, it contains non-numerical value which should be excluded in the following section

In [ ]:
rent_df['median_rent'].unique()

In [ ]:
rent_df = rent_df[(rent_df['median_rent'] != "na") & (rent_df['median_rent'] != "-")]
rent_df['median_rent'].unique()

For individual, we are more concerned at a single room price. The data comes with HDB flat type that suggests the number of rooms available in the flat, and this information will be used to define the average price per room in that particular flat.

Extract number of room from 'flat_type' column

In [ ]:
rent_df['number_of_room'] = [int(flat[0]) for flat in rent_df.flat_type if flat != "EXEC"]
rent_df.head()

Calculate price per room using rental divide by number of rooms

In [ ]:
rent_df['price_per_room'] = rent_df['median_rent'].apply(pd.to_numeric)/rent_df['number_of_room'] 
rent_df.head(15)

In [ ]:
rent_df.groupby(['flat_type']).mean().sort_values('price_per_room', ascending=True)

We can observe that flat_type of higher number of rooms will always result in a cheaper price. Hence, it is advisable for students to group together and rent a HDB of more rooms.

## Exploring and Visualizing Data

Exploring the data by analyzing its statistics and visualizing the values of features and correlations between different features. Explaining the process and the results

let's now combine room type in our towns for a rich visualization we can make

In [ ]:
rent_df_grouped = rent_df.groupby(['town']).mean().sort_values('price_per_room', ascending=True).reset_index()
rent_df_grouped = rent_df_grouped.drop('number_of_room', axis = 1)
rent_df_grouped.head()

Annotate the towns with latitude and longitude generated with geolocator library

In [ ]:
geolocator = Nominatim(user_agent="foursquare_agent")

lat = []
long = []
for town in rent_df_grouped.town:
    location = geolocator.geocode(town + ", SINGAPORE")
    print(town, location.latitude, location.longitude)
    lat.append(location.latitude)
    long.append(location.longitude)

In [ ]:
rent_df_grouped['latitude'] = pd.Series(lat)
rent_df_grouped['longitude'] = pd.Series(long)
rent_df_grouped

Good! Now we are ready to generate a choropleth map of Singapore to visualise the average rental price in each town

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate Singapore map

# generate choropleth map using the rental price of each town in Singapore
venues_map.choropleth(
    geo_data='Singapore.geojson',
    data=rent_df_grouped,
    columns=['town', 'price_per_room'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Rental price per HDB room in Singapore'
)


# add the town center as blue circle markers
for lat, lng, label in zip(rent_df_grouped.latitude, rent_df_grouped.longitude, rent_df_grouped.town):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# display map
venues_map

Notice from the above map, some points are misplaced perhaps due to inaccruate lat long conversion. The points are 'KALLANG/WHAMPOA' and 'CENTRAL'. Special treatment will be placed onto the particular cell to recalculate its location.

In [ ]:
rent_df_grouped.at[21,'town'] = 'KALLANG'

location = geolocator.geocode("KALLANG, SINGAPORE")
rent_df_grouped.at[21,'latitude'] = location.latitude
rent_df_grouped.at[21,'longitude'] = location.longitude

In [ ]:
rent_df_grouped.at[24,'town'] = 'DOWNTOWN CORE'

location = geolocator.geocode("DOWNTOWN CORE, SINGAPORE")
rent_df_grouped.at[24,'latitude'] = location.latitude
rent_df_grouped.at[24,'longitude'] = location.longitude

rent_df_grouped

Now, regenerate the choropleth map with the new lat long information

In [ ]:
latitude, longitude = 1.35, 103.82
sg_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate Singapore map
# generate choropleth map using the rental price of each town in Singapore
sg_map.choropleth(
    geo_data='Singapore.geojson',
    data=rent_df_grouped,
    columns=['town', 'price_per_room'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Rental price per HDB room in Singapore'
)

# add the town center as blue circle markers
for lat, lng, label in zip(rent_df_grouped.latitude, rent_df_grouped.longitude, rent_df_grouped.town):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sg_map)

# display map
sg_map

We are now able to visualise that the highest rental price town mostly locate in the South region of Singapore. The more outskirt from the central it is, the cheaper the rental price will be! Now we can proceed to explore the restaurant and supermarket in the individual towns.

In [ ]:
print("Number of towns in the dataframe is %d" %len(rent_df_grouped))

## Exploring the town with Foursquare API

Let's see how many supermarkets are located in each town by calling the search query of foursquare API. We design the limit as 100 venues and radius of 2km for each town from their calculated latitude and longitude informations.

In [ ]:
search_query = 'supermarket'
radius = 2000
LIMIT = 100

number_of_supermarket = []
supermarket_location = {}
for lat,lng,town in zip(rent_df_grouped.latitude, rent_df_grouped.longitude, rent_df_grouped.town):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius,LIMIT)

    #call the query
    results = requests.get(url).json()

    # assign relevant part of JSON to venues
    supermarket = results['response']['venues']

    # tranform venues into a dataframe
    sup_df = json_normalize(supermarket)
    
    # clean column names by keeping only last term
    sup_df.columns = [column.split('.')[-1] for column in sup_df.columns]

    #store number of supermarket
    number_of_supermarket.append(len(sup_df))
    supermarket_location[town] = sup_df[['lat','lng']].values.tolist()

Now we can append the number of supermarket in each town into the main datafram

In [ ]:
rent_df_grouped['number_of_supermarket'] = pd.Series(number_of_supermarket)
rent_df_grouped.head()

In [ ]:
rent_df_grouped.sort_values('number_of_supermarket', ascending=False)

We can also visualise the distribution of supermarket in each town. Now, let's replot the choropleth map.

In [ ]:
latitude, longitude = 1.35, 103.82
sg_map_w_supermarket = folium.Map(location=[latitude, longitude], zoom_start=12) # generate Singapore map
# generate choropleth map using the rental price of each town in Singapore
sg_map_w_supermarket.choropleth(
    geo_data='Singapore.geojson',
    data=rent_df_grouped,
    columns=['town', 'price_per_room'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Rental price per HDB room in Singapore'
)

# add the town center as blue circle markers
for lat, lng, label in zip(rent_df_grouped.latitude, rent_df_grouped.longitude, rent_df_grouped.town):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sg_map_w_supermarket)

# add the supermarket as red circle markers
for key,value in supermarket_location.items():
    for location in value:
        folium.CircleMarker(
            [location[0], location[1]],
            radius=5,
            color='green',
            popup= 'supermarket',
            fill = False,
            fill_color='green',
            fill_opacity=0.3
    ).add_to(sg_map_w_supermarket)


# display map
sg_map_w_supermarket

Notice that the plotted supermarket marker are not very useful in this sense. However, when one make decision on a specific location, the user can visualise how the supermarket has spreaded in the town and determine which street should him/her look for when finding a place to rent.

Repeat the query with restaurant/food using Foursquare API. Instead of looking at the number of restaurant, we are more interested at the variety of restaurant and how affordable it is. As we have limited premium call to examine the venues, for each town we only take a sample of 10 restaurant id.

In [ ]:
search_query = 'food'
radius = 2000
LIMIT = 100

food_id = {}

for lat,lng,town in zip(rent_df_grouped.latitude, rent_df_grouped.longitude, rent_df_grouped.town):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius,LIMIT)

    #call the query
    results = requests.get(url).json()

    # assign relevant part of JSON to venues
    food = results['response']['venues']

    # tranform venues into a dataframe
    food_df = json_normalize(food)
    
    # clean column names by keeping only last term
    food_df.columns = [column.split('.')[-1] for column in food_df.columns]

    #store 10 restaurant id
    food_id[town] = food_df[['id']].values.tolist()[:10]

Query the details of venue using the id stored earlier. Store the result in new dictionary

In [ ]:
venue_detail = {}
for key,value in food_id.items():
    lst = []
    for id in value:
        venue_id = id[0]
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(url).json()
        lst.append(result)
        
    venue_detail[key] = lst

Due to limited premium call, we only able to examine up to 5 towns of similar rental price range, which are SEMBAWANG, PUNGGOL, CHUA CHO KANG, WOODLANDS and SENGKANG. Assuming the weightage of price, variety and rating is equal, we can determine the food_worthiness of each town by taking their averages.

In [ ]:
food_worthiness = []
for key,value in venue_detail.items():
    price_list  =[]
    rating_list =[]
    category_set = set()
    for result in value:
        try:
            price = result['response']['venue']['price']['tier']
            price_list.append(price) 
        except:

            try:
                rating = result['response']['venue']['rating']
                rating_list.append(rating)
            except:
                try: 
                    category_set.add(result['response']['venue']['categories'][0]['name'])
                except:
                    pass



    food_worthiness.append(((sum(price_list)/len(price_list)) + (sum(rating_list)/len(rating_list)) + (len(category_set)))/3)
    if len(food_worthiness) ==5:
        break

Add food_worthiness column into the 5 towns

In [ ]:
rent_slice_df_grouped = rent_df_grouped.head(5)
rent_slice_df_grouped['food_worthiness'] = pd.Series(food_worthiness)
rent_slice_df_grouped

Normalised the number of supermarket and food_worthiness and generate a last column called decision factor

In [ ]:
supermarket_index = (rent_slice_df_grouped['number_of_supermarket'] - min(rent_slice_df_grouped['number_of_supermarket']))/(max(rent_slice_df_grouped['number_of_supermarket'])- min(rent_slice_df_grouped['number_of_supermarket']))

food_index = (rent_slice_df_grouped['food_worthiness'] - min(rent_slice_df_grouped['food_worthiness']))/(max(rent_slice_df_grouped['food_worthiness'])- min(rent_slice_df_grouped['food_worthiness']))

rent_slice_df_grouped['decision_factor'] = supermarket_index+food_index
rent_slice_df_grouped.sort_values('decision_factor',ascending=False)

Hence, the winner is PUNNGOL who have high number of supermarket and high food worthiness index!

In [ ]:
rent_slice_df_grouped.plot(kind='bar', x= 'town',y='decision_factor')

In [ ]:
latitude, longitude = 1.35, 103.82
sg_map_winner = folium.Map(location=[latitude, longitude], zoom_start=12) # generate Singapore map
# generate choropleth map using the rental price of each town in Singapore
sg_map_winner.choropleth(
    geo_data='Singapore.geojson',
    data=rent_slice_df_grouped,
    columns=['town', 'decision_factor'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Decision index of 5 towns in Singapore'
)

# add the town center as blue circle markers
for lat, lng, label in zip(rent_slice_df_grouped.latitude, rent_slice_df_grouped.longitude, rent_slice_df_grouped.town):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sg_map_winner)


# display map
sg_map_winner